In [2]:
import numpy as np
import h5py
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding3D, BatchNormalization, Flatten, Conv3D, MaxPooling3D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [ ]:
def UNet3D(input_shape = (32, 32, 32, 1)):
    """
    Implementaton of the 3D U-Net the following architecture:
    Input -> CONV3D -> CONV3D -> CONV3D* -> MAXPOOL -> CONV3D -> 
    CONV3D -> CONV3D -> CONV3D -> (DECONV3D + CONV3D*) -> CONV3D -> 
    CONV3D -> CONV3D -> CONV3D -> CONV3D -> Output
    
    Arguments:
    input_shape -- shape of the voxel images of the dataset
    
    Returns:
    model -- a Model() instance in Keras
    """
    
    X_input = Input(input_shape)
    
    X = ZeroPadding3D((1, 1))(X_input)
    
    X = Conv3D(128, (3, 3, 3), strides = (1, 1, 1), name = 'conv1', padding = 'same', 
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = -1, name = 'bn1')(X)
    X = Activation('relu')(X)
    X = Conv3D(128, (3, 3, 3), strides = (1, 1, 1), name = 'conv2', padding = 'same',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = -1, name = 'bn2')(X)    
    X = Activation('relu')(X)
    X = Conv3D(128, (3, 3, 3), strides = (1, 1, 1), name = 'conv3', padding = 'same',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = -1, name = 'bn3')(X)
    X = Activation('relu')(X)
    
    X_down = X
    
    X_down = MaxPooling3D((2, 2, 2), strides=(2, 2, 2))(X_down)
    X_down = Conv3D(128, (3, 3, 3), strides = (1, 1, 1), name = 'conv4', padding = 'same',
                    kernel_initializer = glorot_uniform(seed=0))(X_down)
    X = BatchNormalization(axis = -1, name = 'bn4')(X)    
    X_down = Activation('relu')(X_down)
    X_down = Conv3D(128, (3, 3, 3), strides = (1, 1, 1), name = 'conv5', padding = 'same',
                    kernel_initializer = glorot_uniform(seed=0))(X_down)
    X = BatchNormalization(axis = -1, name = 'bn5')(X)
    X_down = Activation('relu')(X_down)
    X_down = Conv3D(128, (3, 3, 3), strides = (1, 1, 1), name = 'conv6', padding = 'same',
                    kernel_initializer = glorot_uniform(seed=0))(X_down)
    X = BatchNormalization(axis = -1, name = 'bn6')(X)
    X_down = Activation('relu')(X_down)
    X_down = Conv3D(128, (3, 3, 3), strides = (1, 1, 1), name = 'conv7', padding = 'same',
                    kernel_initializer = glorot_uniform(seed=0))(X_down)
    X = BatchNormalization(axis = -1, name = 'bn7')(X)
    X_down = Activation('relu')(X_down)
    
    X_down = UpSmpling3D((2, 2, 2))(X_down)
    
    
    X = Add()([X, X_down])
    
    X = Conv3D(256, (3, 3, 3), strides = (1, 1, 1), name = 'conv8', padding = 'same',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = -1, name = 'bn8')(X)
    X = Activation('relu')(X)

    X = Conv3D(128, (3, 3, 3), strides = (1, 1, 1), name = 'conv9', padding = 'same',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = -1, name = 'bn9')(X)
    X = Activation('relu')(X)

    X = Conv3D(128, (3, 3, 3), strides = (1, 1, 1), name = 'conv10', padding = 'same',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = -1, name = 'bn10')(X)
    X = Activation('relu')(X)
    
    X = Conv3D(128, (3, 3, 3), strides = (1, 1, 1), name = 'conv11', padding = 'same',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = -1, name = 'bn11')(X)
    X = Activation('relu')(X)

    X = Conv3D(1, (1, 1, 1), strides = (1, 1, 1), name = 'conv12', padding = 'same',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = -1, name = 'bn12')(X)
    X = Activation('sigmoid')(X)
    
    model = Model(inputs=[X_input], outputs=[X])
    
    model.compile()
        
    return model
        

# Objective functions

In [3]:
path = '/media/haein/DATA/FlyEM/fib_cube.hdf5'

h = h5py.File(path, 'r')


In [ ]:
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
return 1. - score


In [7]:
def get_masks(y_pred):
    classes = np.unique(y_pred)
    masks = []
    for i in classes:
        masks.append(np.where(y_pred==i, 1, 0))
        
    return masks
        

In [4]:
arr = h['groundtruth'][0]

In [10]:
masks = get_masks(arr)
len(masks)

5

In [5]:
uniq = np.unique(arr)
uniq

array([  96, 7131, 7463, 8053, 9423], dtype=uint64)

In [37]:
mask = []

for i in uniq:
    mask.append(np.where(arr == i, 1, 0))
    
mask[1].shape

(32, 32, 32)

In [28]:
num_clusters = len(uniq)
num_elements = np.where(arr == 96, 1, 0).sum()

In [29]:
num_elements.sum()

3974

In [31]:
z, y, x = np.where(arr==96)

In [33]:
len(z), len(y), len(x)

(3974, 3974, 3974)

In [ ]:
for i in len(z):
    

In [17]:
sample = arr[:,:, 0]
sample.max()

9423